In [1]:
import taichi as ti
import argparse
import time
import numpy as np
import cv2
import os
from imageio import imread, imwrite

[Taichi] version 1.7.3, llvm 15.0.7, commit 5ec301be, osx, python 3.10.18


[I 06/19/25 23:24:59.953 9925599] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
real = ti.f32
ti.init(default_fp=real)

n_grid = 110
dx = 1.0 / n_grid
num_iterations_gauss_seidel = 6
p_dims = num_iterations_gauss_seidel + 1
steps = 100
learning_rate = 100

scalar = lambda: ti.field(dtype=real)
vector = lambda: ti.Vector.field(2, dtype=real)

v = vector()
div = scalar()
p = scalar()
v_updated = vector()
target = scalar()
smoke = scalar()
loss = scalar()

ti.root.dense(ti.i, steps * p_dims).dense(ti.jk, n_grid).place(p)
ti.root.dense(ti.i, steps * p_dims).dense(ti.jk, n_grid).place(p.grad)
block = ti.root.dense(ti.i, steps)

[Taichi] Starting on arch=arm64


In [3]:
def soa(x):
    if isinstance(x, ti.ScalarField):
        block.dense(ti.jk, n_grid).place(x)
        # block.dense(ti.ij, n_grid).place(x.grad)
    else:
        for i in range(x.n):
            soa(x.get_scalar_field(i))

In [4]:
soa(v)
soa(v_updated)
soa(smoke)
soa(div)
ti.root.dense(ti.ij, n_grid).place(target)
ti.root.place(loss)
ti.root.lazy_grad()

In [5]:
# Integer modulo operator for positive values of n
@ti.func
def imod(n, divisor):
    ret = n % divisor
    if ret < 0:
        ret += divisor
    return ret


@ti.func
def dec_index(index):
    new_index = index - 1
    if new_index < 0:
        new_index = n_grid - 1
    return new_index


@ti.func
def inc_index(index):
    new_index = index + 1
    if new_index >= n_grid:
        new_index = 0
    return new_index


@ti.kernel
def compute_div(t: ti.i32):
    for y, x in ti.ndrange(n_grid, n_grid):
        div[t, y, x] = -0.5 * dx * (v_updated[t, inc_index(y), x][0] -
                                    v_updated[t, dec_index(y), x][0] +
                                    v_updated[t, y, inc_index(x)][1] -
                                    v_updated[t, y, dec_index(x)][1])


@ti.kernel
def compute_p(t: ti.i32, k: ti.template()):
    for y, x in ti.ndrange(n_grid, n_grid):
        a = k + t * num_iterations_gauss_seidel
        next_p = (div[t, y, x] + \
                  p[a, dec_index(y), x] +
                  p[a, inc_index(y), x] +
                  p[a, y, dec_index(x)] + \
                  p[a, y, inc_index(x)]) / 4.0
        p[a + 1, y, x] = next_p


@ti.kernel
def update_v(t: ti.i32):
    for y in range(n_grid):
        for x in range(n_grid):
            a = num_iterations_gauss_seidel * t - 1
            v[t, y, x][0] = v_updated[t, y, x][0] - 0.5 * (
                p[a, inc_index(y), x] - p[a, dec_index(y), x]) / dx
            v[t, y, x][1] = v_updated[t, y, x][1] - 0.5 * (
                p[a, y, inc_index(x)] - p[a, y, dec_index(x)]) / dx


@ti.kernel
def advect(field: ti.template(), field_out: ti.template(),
           t_offset: ti.template(), t: ti.i32):
    """Move field smoke according to x and y velocities (vx and vy)
     using an implicit Euler integrator."""
    for y in range(n_grid):
        for x in range(n_grid):
            center_x = y - v[t + t_offset, y, x][0]
            center_y = x - v[t + t_offset, y, x][1]

            # Compute indices of source cell
            left_ix = ti.cast(ti.floor(center_x), ti.i32)
            top_ix = ti.cast(ti.floor(center_y), ti.i32)

            rw = center_x - left_ix  # Relative weight of right-hand cell
            bw = center_y - top_ix  # Relative weight of bottom cell

            # Wrap around edges
            # TODO: implement mod (%) operator
            left_ix = imod(left_ix, n_grid)
            right_ix = inc_index(left_ix)
            top_ix = imod(top_ix, n_grid)
            bot_ix = inc_index(top_ix)

            # Linearly-weighted sum of the 4 surrounding cells
            field_out[t, y, x] = (1 - rw) * (
                (1 - bw) * field[t - 1, left_ix, top_ix] +
                bw * field[t - 1, left_ix, bot_ix]) + rw * (
                    (1 - bw) * field[t - 1, right_ix, top_ix] +
                    bw * field[t - 1, right_ix, bot_ix])


@ti.kernel
def compute_loss():
    for i, j in ti.ndrange(n_grid, n_grid):
        v = (target[i, j] - smoke[steps - 1, i, j])**2 * (1 / n_grid**2)
        ti.atomic_add(loss[None], v)

In [6]:
@ti.kernel
def apply_grad():
    # gradient descent
    for i, j in ti.ndrange(n_grid, n_grid):
        v[0, i, j] -= learning_rate * v.grad[0, i, j]


@ti.ad.no_grad
@ti.kernel
def copy_smoke(t: ti.i32, arr: ti.types.ndarray()):
    for i, j in ti.ndrange(n_grid, n_grid):
        arr[i, j] = smoke[t, i, j]


def forward(output=None):
    T = time.time()
    for t in range(1, steps):
        advect(v, v_updated, -1, t)

        compute_div(t)
        for k in range(num_iterations_gauss_seidel):
            compute_p(t, k)

        update_v(t)
        advect(smoke, smoke, 0, t)

        if output:
            os.makedirs(output, exist_ok=True)
            smoke_ = np.zeros(shape=(n_grid, n_grid), dtype=np.float32)
            copy_smoke(t, smoke_)
            cv2.imshow('smoke', smoke_)
            cv2.waitKey(1)
            cv2.imwrite("{}/{:04d}.png".format(output, t), 255 * smoke_)
            #filename = f"{output}/{t:04d}.png"
            #print(f"Saving frame {t} to {filename}")
            #cv2.imwrite(filename, 255 * smoke_)
    compute_loss()
    print('forward time', (time.time() - T) * 1000, 'ms')

In [7]:
# Load initial and target images
init_img = imread("images/init_smoke.png")[:, :, 0] / 255.0
target_img = imread("images/peace.png")[::2, ::2, 3] / 255.0

for i in range(n_grid):
    for j in range(n_grid):
        smoke[0, i, j] = init_img[i, j]
        target[i, j] = target_img[i, j]

# Training loop
for opt in range(20):  # Fewer iterations for Jupyter
    loss[None] = 0
    with ti.ad.Tape(loss):
        forward("test" if opt % 5 == 0 else None)
    print(f"Iter {opt}, Loss = {loss[None]:.6f}")
    apply_grad()

forward("output")


/var/folders/vw/2yrdy95s5q71dwfz0mwsvmrm0000gn/T/ipykernel_42583/3344461117.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  init_img = imread("images/init_smoke.png")[:, :, 0] / 255.0
/var/folders/vw/2yrdy95s5q71dwfz0mwsvmrm0000gn/T/ipykernel_42583/3344461117.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  target_img = imread("images/peace.png")[::2, ::2, 3] / 255.0
[ WARN:0@1.033] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


forward time 1939.7690296173096 ms
Iter 0, Loss = 0.397772
forward time 50.4000186920166 ms
Iter 1, Loss = 0.278562
forward time 47.90997505187988 ms
Iter 2, Loss = 0.200102
forward time 47.556161880493164 ms
Iter 3, Loss = 0.167054
forward time 49.91507530212402 ms
Iter 4, Loss = 0.148223
forward time 1298.440933227539 ms
Iter 5, Loss = 0.136898
forward time 69.08297538757324 ms
Iter 6, Loss = 0.131735
forward time 48.55704307556152 ms
Iter 7, Loss = 0.124110
forward time 56.82992935180664 ms
Iter 8, Loss = 0.110144
forward time 50.315141677856445 ms
Iter 9, Loss = 0.103662
forward time 1377.934217453003 ms
Iter 10, Loss = 0.093507
forward time 52.22678184509277 ms
Iter 11, Loss = 0.092332
forward time 47.80912399291992 ms
Iter 12, Loss = 0.088864
forward time 49.894094467163086 ms
Iter 13, Loss = 0.087299
forward time 62.8657341003418 ms
Iter 14, Loss = 0.082820
forward time 1329.0550708770752 ms
Iter 15, Loss = 0.077776
forward time 46.65207862854004 ms
Iter 16, Loss = 0.073248
forw

In [8]:
import imageio.v2 as imageio
import os

image_folder = 'output'
image_files = sorted([
    os.path.join(image_folder, fname)
    for fname in os.listdir(image_folder)
    if fname.endswith('.png')
])

gif_filename = 'smoke_simulation.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.05) as writer:
    for filename in image_files:
        image = imageio.imread(filename)
        writer.append_data(image)

print(f"Saved animation to {gif_filename}")


Saved animation to smoke_simulation.gif
